In [1]:
#paths and polynomial ring

def number_of_variables(n):
    global path_ideals
    global path_GB
    global path_JB
    global path_moves
    global path_dataset
    path_ideals =  "/Users/mahshid/Documents/ML_Algebra/Database/{}/{}_Ideals/".format(n,n)
    path_GB =      "/Users/mahshid/Documents/ML_Algebra/Database/{}/{}_Grobner_Basis/".format(n,n)
    path_JB =      "/Users/mahshid/Documents/ML_Algebra/Database/{}/{}_Janet_Basis/".format(n,n)
    path_moves =   "/Users/mahshid/Documents/ML_Algebra/Database/{}/{}_Moves/".format(n,n)
    path_dataset = "/Users/mahshid/Documents/ML_Algebra/Database/{}/{}_Datasets/".format(n,n)

    if n == 4:
        P.<X1, X2, X3, X4> = PolynomialRing(QQ, 4, order = 'degrevlex')  
    elif n == 5:
        F = FiniteField( 30011)
        #F = QQ
        P.<X1, X2, X3, X4, X5> = PolynomialRing(F, 5, order = 'degrevlex')
    elif n == 6:
        F = FiniteField(30011)
        P.<X1, X2, X3, X4, X5, X6> = PolynomialRing(F, 6, order = 'degrevlex')
    return P    

In [2]:
# All the functions for creating statistical features
def subset_n(n):
    Subsets = []
    if n==1:
        Subsets.append([1])
    else:
        temp = subset_n(n-1)
        Subsets += temp
        Subsets.append([n])
        for i in range(len(temp)):
            Subsets.append(temp[i]+[n])
    return Subsets

def str_sub(label, n):
    labels = []
    for sub in subset_n(n):
        temp = "{}_with".format(label)
        for lb in sub:
            temp += "_X{}".format(lb)
        labels.append(temp)
    return labels

def func_sub(func, F, V, n):
    funcs = []
    for sub in subset_n(n):
        temp = [V[i-1] for i in sub]
        funcs.append(func(temp, F))
    return funcs

def func_range(func, F, V, n):
    funcs = []
    for i in range(n):
       # print(type(func))
        funcs.append(func(V[i], F))
    return funcs

def strs_range(strs, n):
    labels = []
    for lb in strs:
        for i in range(1, n+1):
            labels.append("{}_{}".format(lb, i))
    return labels

def Make_columns(N):
    columns = ['#gens', '#Terms',
               'min_deg', 'max_deg', 'av_deg',
               'min_terms', 'max_terms', 'av_terms',
               'deg_sum(F)',
               '#pure_var_power', '#pure_var_power_lt'
               ]
    columns += strs_range(["min_deg_of_X", "max_deg_of_X", "av_deg_of_X",
                           "min_lt_deg_of_X", "max_lt_deg_of_X", "av_lt_deg_of_X",
                           "cls", "cls_of_lt"]
                          , N)
    columns += str_sub("generators", N)
    columns += str_sub("lt", N)
    columns += str_sub("terms", N)
    return columns

def Make_results(F):
    classes = cls_of_terms(F)
    lt_classes = cls_of_lt(F)
    V = list(P.gens())
    N = len(V)
    results = [len(F), num_terms(F),
               min_deg(F), max_deg(F), av_deg(F),
               min_terms(F), max_terms(F), av_terms(F),
               deg_sum(F),
               num_pure_variable(F), num_pure_variable_lt(F)]
    function = [min_deg_of, max_deg_of, av_deg_of,
                min_lt_deg_of, max_lt_deg_of, av_lt_deg_of]
    for func in function:
        results += func_range(func, F, V, N)
    results += classes
    results += lt_classes
    function = [generators_with, generators_lt_with, terms_with]
    for func in function:
        results += func_sub(func, F, V, N)
    
    return results

In [4]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from numpy import random
import math
import tqdm
import time
import numpy as np



def clss(m): #m is a monomial
    d = list(m.degrees())
    d.reverse()
    for i in range(1, len(d)+1):
        if d[i-1]!= 0:
            return i
    return 0
    
##################################################### 

def m_index(m): #m is a monomial
    d = list(m.degrees())
    d.reverse()
    for i in range(1, len(d)+1):
        if d[i-1]!= 0:
            return len(d) - i + 1
    return 0

#####################################################        

def QS_moves(F):
    I = ideal(f for f in F)
    G1 = I.groebner_basis()
    GB = []

    for f in G1:
        GB.append(f.lt())
    J = ideal(f for f in GB)

    degG = GB[0].degree()
    for g in GB:
        if g.degree() > degG:
            degG = g.degree()
    
    num = 0
    L = []   #List of all moves
    for m in GB:
        if m in QQ:
            break
        C = []   #List of all moves for every element
        for i in range(m_index(m)): #finding suitable i for (k,i)
            if P.gens()[i]^degG * m // P.gens()[m_index(m) - 1] ^ m.degrees()[m_index(m)-1] not in J : #and [m_index(m),i+1] not in L
                # x_i ^{deg(G)}*x^{mu}/x_k^{mu_k}
                L.append([m_index(m),i+1])
                C.append([m_index(m),i+1])
                num = num + 1
        #print(C)  
    l1 = []
    l2 = []
    for i in range(len(L)):
        if L[i] not in l1:
            l1.append(L[i])
            l2.append(1)
        
        else:
            for j in range(len(l1)):
                if l1[j] == L[i]:
                    l2[j] += 1
    List = []
    for i in range(len(l1)):
        List.append([l1[i],l2[i]])

    #print(num)
    return List

#####################################################    

def QS_moves2(F):           # F is a Groebner Basis

    GB = LT_I(F)
    
    degG = max_deg(GB)
    J = ideal(f for f in GB)
    
    num = 0
    L = []   #List of all moves
    for m in GB:
        if m in QQ:
            break
        C = []   #List of all moves for every element
        for i in range(m_index(m)): #finding suitable i for (k,i)
            if P.gens()[i]^degG * m // P.gens()[m_index(m) - 1] ^ m.degrees()[m_index(m)-1] not in J : #and [m_index(m),i+1] not in L
                # x_i ^{deg(G)}*x^{mu}/x_k^{mu_k}
                L.append([m_index(m),i+1])
                C.append([m_index(m),i+1])
                num = num + 1
        #print(C)  
    l1 = []
    l2 = []
    for i in range(len(L)):
        if L[i] not in l1:
            l1.append(L[i])
            l2.append(1)
        
        else:
            for j in range(len(l1)):
                if l1[j] == L[i]:
                    l2[j] += 1
    List = []
    for i in range(len(l1)):
        List.append([l1[i],l2[i]])

    #print(num)
    return List


#####################################################     

def Monomials(d):
    if d>1:
        L = Monomials(d-1)
        M = []
        for m in L:
            for v in P.gens():
                if v*m not in M:
                    M.append(v*m)
    if d==1:
        M = list(P.gens())
    return M

#####################################################     

def Polynomial_Transformation(f, move):
    a = move[0] - 1
    b = move[1] - 1
    V = list(P.gens())
    V[a] = V[a] + V[b]
    return f(V)


#####################################################     

def Transformation(F , move):
    new_F = []
    for f in F:
        new_F.append(Polynomial_Transformation(f, move))
    return new_F 


#####################################################  


def Polynomial_Transposition(f, p):
    a = p[0] - 1
    b = p[1] - 1
    V = list(P.gens())
    V[a], V[b] = V[b], V[a]
    return f(V)

#####################################################  


def Transposition(F , p):
    new_F = []
    for f in F:
        new_F.append(Polynomial_Transposition(f, p))
    return new_F 


#####################################################  

def paths(Functions, n):
    F = copy(Functions)
    OPath = []
    QS = QS_moves(F)
    QSC = copy(QS)       #save QS for next Rpath
    while len(QS)!= 0:
        maximum = 0
        for i in range(len(QS)):          
            if QS[i][1] > maximum:
                maximum = QS[i][1]
                r = QS[i][0]
        OPath.append(r)
        F = Transformation(F,r)
        QS = QS_moves(F)
    
    sum = 0
    l = []
    Min = 1000
    Max = 0
    for i in range(n):
        F = copy(Functions)
        QS = copy(QSC)
        RPath = []
        while len(QS)!= 0:
            r = randrange(len(QS))
            RPath.append(QS[r][0])
            F = Transformation(F,QS[r][0])
            QS = QS_moves(F)
        l.append(len(RPath))    
        sum += len(RPath)
        
    return (len(OPath), round(float(sum/n), 1), max(l), min(l))     #return length of our path and average length of random path

#####################################################
def several_randompath(Functions, n):
    sum = 0
    for i in range(n):
        sum += len(randompath(Functions))
    return(sum/n) 

#####################################################

def randompath(Functions):
    F = copy(Functions)
    Path = []
    QS = QS_moves(F)
    while len(QS)!= 0:
        r = randrange(len(QS))
        Path.append(QS[r][0])
        F = Transformation(F,QS[r][0])
        QS = QS_moves(F)
    return Path

#####################################################

def number_of_nonzero(L): #number_of_nonzero elements in list L
    c = 0
    for l in L:
        if l != 0:
            c += 1
    return c      

#####################################################

def Moves_to_Features(Move):
    L = [0 for i in range(16)]
    for M in Move:
        L[4*(M[0][0]-1) + M[0][1] - 1] = M[1]
    QS = []
    for i in range(len(L)):
        if i == 4 or i == 8 or i == 9 or i == 12 or i == 13 or i == 14:
            QS.append(L[i])
    return QS        

In [5]:
# For features

def num_terms(F):
    c = 0
    for f in F:
        while f != 0:
            f = f - f.lt()
            c += 1
    return c    

def min_deg(F):
    min = 10000    # just a large number!
    for f in F:
        if f.degree() < min:
            min = f.degree()
    return min


def max_deg(F):
    max = 0
    for f in F:
        if f.degree() > max:
            max = f.degree()
    return max


def av_deg(F):
    s = 0
    for f in F:
        s += f.degree()
    return round(float(s/len(F)), 1)


def min_terms(F):
    min = 10000    # just a large number!
    for f in F:
        if num_terms([f]) < min:
            min = num_terms([f])
    return min


def max_terms(F):
    max = 0
    for f in F:
        if num_terms([f]) > max:
            max = num_terms([f]) 
    return max


def av_terms(F):
    s = 0
    for f in F:
        s += num_terms([f]) 
    return round(float(s/len(F)), 1)


def min_deg_of(X, F):
    d = []
    for f in F:
        d.append(f.degree(X))
    return min(d)  


def max_deg_of(X, F):
    d = []
    for f in F:
        d.append(f.degree(X))
    return max(d) 


def av_deg_of(X, F):
    d = []
    for f in F:
        d.append(f.degree(X))
    return round(sum(d)/len(d), 1)


def min_lt_deg_of(X, F):
    d = []
    for f in F:
        d.append(f.lt().degree(X))
    return min(d) 


def max_lt_deg_of(X, F):
    d = []
    for f in F:
        d.append(f.lt().degree(X))
    return max(d)  

def av_lt_deg_of(X, F):
    d = []
    for f in F:
        d.append(f.lt().degree(X))
    return round(sum(d)/len(d), 1)


def generators_with(X,F):
    c = 0
    for f in F:
        i = 0
        for x in X:
            if f.degree(x)==0:
                i = 1
                break
        if i == 0:        
            c += 1
    return c


def terms_with(X,F):
    c = 0
    for f in F:
        while f != 0:
            i = 0
            for x in X:
                if f.lt().degree(x)==0:
                    i = 1
                    break
            if i == 0:
                c += 1
            f = f - f.lt()
    return c


def generators_lt_with(X,F):
    c = 0
    for f in F:
        i = 0
        while(True):
            if f.lt().degree(X[i]) == 0:
                break
            i += 1    
            if i == len(X):        
                c += 1
                break
    return c
    
    
def deg_sum(F):
    s = 0
    for f in F:
        s += f.degree()
    return s   




def num_pure_variable_lt(F):
    c = 0
    for f in F:
        L = list(f.lt().degrees())
        if number_of_nonzero(L) == 1:
            c += 1
    return c   

def num_pure_variable(F):
    c = 0
    for f in F:
        while f != 0:
            L = list(f.lt().degrees())
            if number_of_nonzero(L) == 1:
                c += 1
            f -= f.lt()    
    return c   



def cls_of_terms(F):
    L = []
    for _ in range(len(P.gens())):
        L.append(0)
    for f in F:
        fc = copy(f)
        while fc != 0:
            L[clss(fc.lt()) - 1] += 1
            fc -= fc.lt()
    return L   



def cls_of_lt(F):
    L = []
    for _ in range(len(P.gens())):
        L.append(0)
    for f in F:
        L[clss(f.lt()) - 1] += 1
    return L   



def LT_I (H):
    LT = []
    for h in H:
        LT.append(h.lt())
    return LT    

In [6]:
def Random_Ideal(gen_lambda, terms_lambda, deg_lambda):

    gen = random.poisson(lam = gen_lambda, size = 1)[0] + 2
    Ideal = []

# start producing an ideal
    for i in range(gen):
        terms = random.poisson(lam = terms_lambda, size = 1)[0] + 2
        deg = random.poisson(lam = deg_lambda, size = 1)[0] + 1
        P = 0
        while(P == 0):
            P = Random_Poly(deg, terms)
            Ideal.append(P)

    return Ideal




def Random_Poly(deg, terms):
    poly = 0
    for _ in range(terms):
        c = random.choice(list(set([x for x in range(-9, 9)]) - set([0]))) #random coeficient (-9,9)\{0}
        poly += random.choice(Monomials(deg + 2))*c
    return poly    

In [7]:
def min_element(F): # F is list of generators of a monomial ideal
    m = F[0]
    for f in F:
        if f < m:
            m = f
    return m   

def X_P(h):           # Pommaret multiplicative variables
    d = list(h.degrees())
    d.reverse()
    for i in range(len(d)):
        if d[i] != 0:
            break
    return list(P.gens()[3 - i:])    

def X_P_bar(h):
    return [v for v in P.gens() if v not in X_P(h)]


def X_J(h, H):       # Janet multiplicative variables,      h is a monomial in a finite set of monomials H.
    multi_var = []
    
    flag = 1                                      # for checking x_1
    for u in H:
        if u.degrees()[0] > h.degrees()[0]:
            flag = 0
            break
    if flag:
        multi_var.append(P.gens()[0])
        
        
    for i in range(1, len(P.gens())):            # for checking x_i for 1<i<=n
        U1 = U(H, i, h)  
        flag = 1
        for u in U1:
            if u.degrees()[i] > h.degrees()[i]:
                flag = 0
                break
        if flag:
            multi_var.append(P.gens()[i])
    return multi_var    
        
def U(H, i, m):    #U[m_1, ... , m_{i-1}]    i > 1
    L = []
    for h in H:
        m_deg = m.degrees()
        h_deg = h.degrees()
        j = 0
        while True:
            if m_deg[j]!= h_deg[j]:
                break
            if j == i-1:
                L.append(h)
                break
            j += 1    
    return L

def X_J_bar(h, H):
    return [v for v in P.gens() if v not in X_J(h, H)]

In [8]:
def Combination(x , n):   # n should be an integer
    p = 1
    for i in range(n):
        p = p * (x - i)
    return p / math.factorial(n) 


def Volume_Polynomial(H, r):   
    sum = 0
    for h in H:
        kh = len(X_J(h, H))
        qh = h.degree()
        sum += Combination(r - qh + kh - 1 , kh - 1)
    return sum

In [9]:
def Monomial_Completion(G):
    H = copy(G)
    flag = True
    A = []
    while flag:
        S = []
        for h in H:
            for y in X_J_bar(h, H):
                if y*h not in A:
                    S.append(y*h)       
        f = True
        while len(S) != 0 and f:
            s = min_element(S)
            S.remove(s)
            
            for h in H:
                quo, rem = s.quo_rem(h)
                if rem == 0 and set(quo.variables()).issubset(set(X_J(h, H))): #and len(quo.variables()) != 0:
                    f = True
                    break
                else:
                    f = False    
            if not f:
                H.append(s)
                A.append(s)
                flag = False
        if flag:
            return H
        else:
            flag = True

In [10]:
def H_q(H): 
    L = []
    q = max_deg(H)
    for h in H:
        if q - h.degree() > 0:
            M1 = Monomials(q - h.degree())
            for m in M1:
                if set(m.variables()).issubset(set(X_J(h, H))):
                    L.append(h*m)
        elif q == h.degree():
            L.append(h)
            
    return list(set(L))

In [11]:
def Beta_Vec_P2(F):   # For Prommaret span   #OLD! and also WRONG :/
    B = []
    for i in range(len(P.gens())):
        B.append(0)   
    H = H_q(F)
    M1 = Monomials(1)
    H_plus = []
    for h in H:
        for m in M1:
            if set(m.variables()).issubset(set(X_P(h))):
                H_plus.append(h*m)
    H_plus = list(set(H_plus))
    for h in H_plus:
        c = clss(h)
        B[len(P.gens()) - c] += 1
    return B

In [12]:
def Beta_Vec_P(F):   # For Prommaret span   # F needs to be Janet Basis
    B = [0 for i in range(len(P.gens()))]
    H = H_q(F)
    for h in H:
        c = clss(h)
        B[len(P.gens()) - c] += 1
    return B

In [13]:
def Volume_Polynomial_Pommaret(B, r):
    v = 0
    B.reverse()
    for i in range(len(B)):
        v += B[i]* Combination(r + i, i)
    return v    

In [14]:
def Volume_Polynomial_Pommaret2(H, r):
    s = 0
    for h in H:
        d = h.degree()
        s += Combination(r - d + clss(h) - 1, clss(h) - 1)
    return s

In [15]:
def Update(F, s, x):
    H = copy(F)
    UU = U([hh[0] for hh in H], P.gens().index(x) + 1 , s)  # update parts of H
    for u in UU:
        for j in range(len(H)):   
            if u == H[j][0]:
                X = X_J(u,UU)
                X_0_x = [h for h in H[j][2] if h in list(P.gens()[:P.gens().index(x) + 1])]
                XJ = []
                for i in range(len(P.gens())):
                    if i < P.gens().index(x) + 1 and P.gens()[i] in X_0_x:
                        XJ.append(P.gens()[i])
                    if i >= P.gens().index(x) + 1 and P.gens()[i] in X:
                        XJ.append(P.gens()[i]) 
                H[j] = [u, H[j][1], XJ]    
    return H

In [16]:
def Janet_Multiplicative(s, H):
    for h in H:
        quo, rem = s.quo_rem(h)
        if rem == 0 and set(quo.variables()).issubset(set(X_J(h, H))):
            Janet = True
            break
        else:
            Janet = False
    return Janet, h

In [17]:
def Fast_Monomial_Completion_finitefield(F):
    H = []
    for f in F:
        H.append([f, [], X_J(f,F)]) # Line 2
    F = FiniteField(30011)
    P.<X1, X2, X3, X4, X5> = PolynomialRing(F, 5, order = 'lex')   
    H = sorted(H, key=lambda x: x[0])   # Line 3 
    P.<X1, X2, X3, X4, X5> = PolynomialRing(F, 5, order = 'degrevlex') 
    d = H[0][0].degree()

    flag = False                   # Line 4
    while not flag:                # Line 5
        flag = True               # Line 6
        for i  in range(len(H)):                # Condition of Line 7
            f = True
        
            if set(H[i][1]).union(H[i][2]) != set(list(P.gens())):
                f = False
                t = i
                break
                
        if not f:                  # Line 7
            flag = False           # Line 8
            for v in list(P.gens()):             # Line 9
                if v not in list(set(H[t][1]).union(H[t][2])):
                    x = v
                    break     
            H[t][1].append(x)      # Line 10
            s = x * H[t][0]        # condition of Line 11
            Janet, _ = Janet_Multiplicative(s, [h[0] for h in H])         
            if not Janet:              # Line 11
                H.append([s, [], []])      # Line 12
                H[-1] = [s, [], X_J(H[-1][0], [h[0] for h in H])]
                P.<X1, X2, X3, X4, X5> = PolynomialRing(F, 5, order = 'lex')   
                H = sorted(H, key=lambda x: x[0])   # Line 3 
                P.<X1, X2, X3, X4, X5> = PolynomialRing(F, 5, order = 'degrevlex') 
                for h in H:
                    if s == h[0]:
                        s_index = H.index(h)
                        break
                H = Update(H, s, x)
    return [f[0] for f in H]

In [18]:
def Fast_Monomial_Completion_finitefield_lex(F):
    H = []
    for f in F:
        H.append([f, [], X_J(f,F)]) # Line 2  
    F = FiniteField( 30011)    
    P.<X1, X2, X3, X4, X5> = PolynomialRing(F, 5, order = 'lex')    
    H = sorted(H, key=lambda x: x[0])   # Line 3 
    P.<X1, X2, X3, X4, X5> = PolynomialRing(F, 5, order = 'degrevlex')
    d = H[0][0].degree()

    flag = False                   # Line 4
    while not flag:                # Line 5
        flag = True               # Line 6
        for i in range(len(H)):                # Condition of Line 7
            f = True
        
            if set(H[i][1]).union(H[i][2]) != set(list(P.gens())):
                f = False
                t = i
                break
                
        if not f:                  # Line 7
            flag = False           # Line 8
            for v in list(P.gens()):             # Line 9
                if v not in list(set(H[t][1]).union(H[t][2])):
                    x = v
                    break     
            H[t][1].append(x)      # Line 10
            s = x * H[t][0]        # condition of Line 11
            Janet, _ = Janet_Multiplicative(s, [h[0] for h in H])         
            if not Janet:              # Line 11
                H.append([s, [], []])      # Line 12
                H[-1] = [s, [], X_J(H[-1][0], [h[0] for h in H])] 
                P.<X1, X2, X3, X4, X5> = PolynomialRing(F, 5, order = 'lex')
                H = sorted(H, key=lambda x: x[0])   # Line 3 
                P.<X1, X2, X3, X4, X5> = PolynomialRing(F, 5, order = 'degrevlex')
                for h in H:
                    if s == h[0]:
                        s_index = H.index(h)
                        break
                H = Update(H, s, x)
    return [f[0] for f in H]

In [20]:
def Fast_Monomial_Completion(F):
    #FF = QQ 
    #FF = FiniteField( 30011)
    #P.<X1, X2, X3, X4, X5> = PolynomialRing(FF, 5, order = 'lex')
    H = []
    for f in F:
        H.append([f, [], X_J(f,F)]) # Line 2

    H = sorted(H, key=lambda x: x[0])   # Line 3 
    #P.<X1, X2, X3, X4, X5> = PolynomialRing(FF, 5, order = 'degrevlex')
    d = H[0][0].degree()

    flag = False                   # Line 4
    while not flag:                # Line 5
        flag = True               # Line 6
        for i  in range(len(H)):                # Condition of Line 7
            f = True
        
            if set(H[i][1]).union(H[i][2]) != set(list(P.gens())):
                f = False
                t = i
                break
                
        if not f:                  # Line 7
            flag = False           # Line 8
            for v in list(P.gens()):             # Line 9
                if v not in list(set(H[t][1]).union(H[t][2])):
                    x = v
                    break     
            H[t][1].append(x)      # Line 10
            s = x * H[t][0]        # condition of Line 11
            Janet, _ = Janet_Multiplicative(s, [h[0] for h in H])         
            if not Janet:              # Line 11
                H.append([s, [], []])      # Line 12
                H[-1] = [s, [], X_J(H[-1][0], [h[0] for h in H])]
                #P.<X1, X2, X3, X4, X5> = PolynomialRing(FF, 5, order = 'lex')
                H = sorted(H, key=lambda x: x[0])
                #P.<X1, X2, X3, X4, X5> = PolynomialRing(FF, 5, order = 'degrevlex')
                for h in H:
                    if s == h[0]:
                        s_index = H.index(h)
                        break
                H = Update(H, s, x)
    return [f[0] for f in H]

In [22]:
def Best_choice_for_transformation(I):                      # an ideal as the input
    
    All_moves = []                                      # compute all possible moves
    for i in range(2,len(P.gens()) + 1):
        for j in range(1,i):
            All_moves.append([i,j])
    Moves = []        
    for move in All_moves:                              # Choosing a MOVE
        transformation = Transformation(I, move)        # Do the transformation on the ideal   
        Ideal = ideal(f for f in transformation)                        
        GB = Ideal.groebner_basis() 
        JB_new = Fast_Monomial_Completion(LT_I(GB))     # Monomial Janet basis for the ideal after transfrmation
        v = Volume_Polynomial_PommaretSpan(JB_new, P.gens()[0])
        V = Coefficients(v, len(P.gens())-1)
        Moves.append([move, V])
    Moves = sorted(Moves, key=lambda x: x[1])   
    #print(Moves)
        
    return Moves[-1][0]   

In [23]:
def dim(H):
    H = Combination(P.gens()[0] +len(P.gens())-1 , len(P.gens())-1) - Volume_Polynomial(H, P.gens()[0])
    return H.degree()+1

In [24]:
def division(F, f):
    r"""
    Computes division for multivariate polynomials.

    INPUT:

    - ``F`` -- a list of polynomials ``f_0``, ..., ``f_{s-1}``

    - ``f`` -- a polynomial

    OUTPUT:

    A tuple (q, r) such that $f = \sum_{i=0}^{s-1} q_i f_i + r$.
    """
    p = f
    q = []
    r = 0
    for i in range(len(F)):
        q.append(0) # f.parent(0)
    while p != 0:
        i = 1
        divisionoccurred = 0
        while i<=len(F) and divisionoccurred == 0:
            if p.lt() % F[i-1].lt() == 0:
                q[i-1] = q[i-1] + (p.lt().quo_rem(F[i-1].lt())[0])
                (p.lt().quo_rem(F[i-1].lt())[0])
                p = p - (p.lt().quo_rem(F[i-1].lt())[0])*F[i-1]
                divisionoccurred = 1
            else:
                i+=1
        if divisionoccurred == 0:
            r = r + p.lt()
            p = p - p.lt()
    return q,r

In [25]:
def Coefficients(f, deg):
    C = [0 for i in range(deg+1)]
    f = P(f)
    while(f != 0):
        C[f.lt().degree()]  = f.lt().coefficients()[0]
        f -= f.lt()
    C.reverse()    
    return C

In [26]:
def Best_choice_for_transformation2(I):                      # an ideal as the input
    
    All_moves = []                                      # compute all possible moves
    for i in range(2,len(P.gens()) + 1):
        for j in range(1,i):
            All_moves.append([i,j])
    Beta_vectors = []        
    for move in All_moves:                              # Choosing a MOVE
        transformation = Transformation(I, move)        # Do the transformation on the ideal   
        Ideal = ideal(f for f in transformation)                        
        GB = Ideal.groebner_basis() 
        JB_new = Fast_Monomial_Completion(LT_I(GB))     # Monomial Janet basis for the ideal after transfrmation
        B = new_Beta_Vec_P(JB_new)
        Beta_vectors.append([move, B])
    Beta_vectors = sorted(Beta_vectors, key=lambda x: x[1])   
    #print(Moves)
        
    return Beta_vectors[-1][0]    

In [27]:
def Volume_Polynomial_Pommaret2(H, r):
    s = 0
    for h in H:
        d = h.degree()
        s += Combination(r - d + clss(h) - 1, clss(h) - 1)
    return s

In [28]:
def Method2(I):
    All_moves = []                                      # compute all possible moves
    for i in range(2,len(P.gens()) + 1):
        for j in range(1,i):
            All_moves.append([i,j])
    Moves = []        
    for move in All_moves:                              # Choosing a MOVE
        transformation = Transformation(I, move)        # Do the transformation on the ideal   
        Ideal = ideal(f for f in transformation)                        
        GB = Ideal.groebner_basis() 
        H = Fast_Monomial_Completion(LT_I(GB))     # Monomial Janet basis for the ideal after transfrmation
        v = Volume_Polynomial_Pommaret2(H, P.gens()[0])
        V = Coefficients(v, len(P.gens())-1)
        Moves.append([move, V])
    Moves = sorted(Moves, key=lambda x: x[1])      
    return  All_moves.index( Moves[-1][0])+1 

In [29]:
def Sigma(H , B):
    s = 0
    BP = Beta_Vec_P(H)
    BP.reverse()
    B.reverse()
    for k in range(len(P.gens())):
        s += (k + 2) *(B[k] - BP[k])
    return s

In [30]:
def Method4(I):
    All_moves = []                                      # compute all possible moves
    for i in range(2,len(P.gens()) + 1):
        for j in range(1,i):
            All_moves.append([i,j]) 
    Moves = []        
    for move in All_moves:                              # Choosing a MOVE
        transformation = Transformation(I, move)        # Do the transformation on the ideal   
        Ideal = ideal(f for f in transformation)                        
        GB = Ideal.groebner_basis() 
        H = Fast_Monomial_Completion(LT_I(GB))     # Monomial Janet basis for the ideal after transfrmation
        B = Beta_Vector(H)
        s = Sigma(H , B)
        Moves.append([move, s])
    Moves = sorted(Moves, key=lambda x: x[1])   
        
    return  Moves 

In [31]:
def Beta_V(V, q):  # Only for 4 variables ; V is a list of coefficients and d is the degree of Janet basis
    B4 = 6 * V[0]
    B3 = 2*(3*V[0]*q + V[1] - B4)
    B2 = 3*V[0]*q^2 + 2*V[1]*q + V[2] - 3/2 * B3 - 11 * B4/6
    B1 = V[0]*q^3 + V[1]*q^2 + V[2]*q + V[3] - B2 - B3 - B4
    return [B4,B3,B2,B1]